## MongoDB Python Basics

Connect to MongoDB

In [1]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")

Create a database (not technically created until data actually added)

In [11]:
dbDev = client["Dev"]
client.list_database_names()  # get list of all databases

['Dev', 'admin', 'config', 'local']

Create a collection in the database (like a "table" but not really since we are NoSQL, more accurately a container for JSON data)

In [6]:
col = dbDev["collection"]
col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Dev'), 'collection')

Insert single record

In [16]:
mydict = { "artist": "Bob Dylan", "album": "Time Out of Mind", "year": 1997 }

x = col.insert_one(mydict)

In [17]:
x.inserted_id  # unique ID assigned by MongoDB (can assign your own)
# use following in each record to assigne your own ids: "_id": <id #>

ObjectId('5cbfe413c0f9820c583986dd')

Insert multiple records

In [13]:
albums = \
[{"artist": "Bob Dylan", "album": "Highway 61 Revisited", "year": 1965},
 {"artist": "Bob Dylan and The Band", "album": "The Basement Tapes", "year": 1975},
 {"artist": "Bob Dylan", "album": "Infidels", "year": 1983},
 {"artist": "Bob Dylan", "album": "Blonde on Blonde", "year": 1966},
 {"artist": "Bob Dylan and The Grateful Dead", "album": "Dylan & The Dead", "year": 1989}]


In [14]:
x = col.insert_many(albums)
x.inserted_ids

[ObjectId('5cbfe398c0f9820c583986d8'),
 ObjectId('5cbfe398c0f9820c583986d9'),
 ObjectId('5cbfe398c0f9820c583986da'),
 ObjectId('5cbfe398c0f9820c583986db'),
 ObjectId('5cbfe398c0f9820c583986dc')]

"Querying"

In [18]:
col.find_one()  # get first record ("document") in collection

{'_id': ObjectId('5cbfe1e1c0f9820c583986d7'),
 'album': 'Time Out of Mind',
 'artist': 'Bob Dylan',
 'year': 1997}

In [20]:
[x for x in col.find()]  # get all documents in collection

[{'_id': ObjectId('5cbfe1e1c0f9820c583986d7'),
  'album': 'Time Out of Mind',
  'artist': 'Bob Dylan',
  'year': 1997},
 {'_id': ObjectId('5cbfe398c0f9820c583986d8'),
  'album': 'Highway 61 Revisited',
  'artist': 'Bob Dylan',
  'year': 1965},
 {'_id': ObjectId('5cbfe398c0f9820c583986d9'),
  'album': 'The Basement Tapes',
  'artist': 'Bob Dylan and The Band',
  'year': 1975},
 {'_id': ObjectId('5cbfe398c0f9820c583986da'),
  'album': 'Infidels',
  'artist': 'Bob Dylan',
  'year': 1983},
 {'_id': ObjectId('5cbfe398c0f9820c583986db'),
  'album': 'Blonde on Blonde',
  'artist': 'Bob Dylan',
  'year': 1966},
 {'_id': ObjectId('5cbfe398c0f9820c583986dc'),
  'album': 'Dylan & The Dead',
  'artist': 'Bob Dylan and The Grateful Dead',
  'year': 1989},
 {'_id': ObjectId('5cbfe413c0f9820c583986dd'),
  'album': 'Time Out of Mind',
  'artist': 'Bob Dylan',
  'year': 1997}]

In [30]:
# return all album names, no id (1/0 is T/F for show)
[x for x in col.find({},{"_id":0, "album":1})]  

[{'album': 'Time Out of Mind'},
 {'album': 'Highway 61 Revisited'},
 {'album': 'The Basement Tapes'},
 {'album': 'Infidels'},
 {'album': 'Blonde on Blonde'},
 {'album': 'Dylan & The Dead'},
 {'album': 'Time Out of Mind'}]

Delete a record

In [43]:
# need this to query by ObjectId, Binary JSON encoding library
from bson.objectid  import ObjectId  
col.find_one({"_id": ObjectId('5cbfe413c0f9820c583986dd')})

{'_id': ObjectId('5cbfe413c0f9820c583986dd'),
 'album': 'Time Out of Mind',
 'artist': 'Bob Dylan',
 'year': 1997}

In [51]:
x = col.delete_one({"_id": ObjectId('5cbfe413c0f9820c583986dd')})
[x for x in col.find({},{"_id":0, "album":1})]  
# note that dupe record no longer exists

# can also delete:
# col.delete_many({some query})
# col.delete_many({})  # delete all docs in a collection
# col.drop()  # drop the collection

[{'album': 'Time Out of Mind'},
 {'album': 'Highway 61 Revisited'},
 {'album': 'The Basement Tapes'},
 {'album': 'Infidels'},
 {'album': 'Blonde on Blonde'},
 {'album': 'Dylan & The Dead'}]

More querying

In [53]:
# query w/ a dict
query = {"album": "Blonde on Blonde"}
[doc for doc in col.find(query)]

[{'_id': ObjectId('5cbfe398c0f9820c583986db'),
  'album': 'Blonde on Blonde',
  'artist': 'Bob Dylan',
  'year': 1966}]

In [68]:
# get Dylan in the 80s
query = {"$and":[{"year":{"$gte":1980}}, {"year":{"$lt":1990}}]}
[doc for doc in col.find(query).sort("year", -1)]  #sort desc

# note: 
# $and, $or
# $gt >, $gte >=
# $lt <, $lte <=


[{'_id': ObjectId('5cbfe398c0f9820c583986dc'),
  'album': 'Dylan & The Dead',
  'artist': 'Bob Dylan and The Grateful Dead',
  'year': 1989},
 {'_id': ObjectId('5cbfe398c0f9820c583986da'),
  'album': 'Infidels',
  'artist': 'Bob Dylan',
  'year': 1983}]

In [76]:
# top 2 albums
[x for x in col.find().limit(2)]

[{'_id': ObjectId('5cbfe1e1c0f9820c583986d7'),
  'album': 'Time Out of Mind',
  'artist': 'Bob Dylan',
  'year': 1997},
 {'_id': ObjectId('5cbfe398c0f9820c583986d8'),
  'album': 'Highway 61 Revisited',
  'artist': 'Bob Dylan',
  'year': 1965}]

In [66]:
# albums made with The Band
query = {"artist": {"$regex": "[tT]he [B]and"}}
[doc for doc in col.find(query)]


[{'_id': ObjectId('5cbfe398c0f9820c583986d9'),
  'album': 'The Basement Tapes',
  'artist': 'Bob Dylan and The Band',
  'year': 1975}]

Update/Add field to record(s)

In [73]:
# update (add length to The Basement Tapes)
query = {"artist": {"$regex": "[tT]he [B]and"}}
update = {"$set":{"length (min)": 77 }}
x = col.update_one(query, update)
[doc for doc in col.find(query)]

# use .update_many() to update many records

[{'_id': ObjectId('5cbfe398c0f9820c583986d9'),
  'album': 'The Basement Tapes',
  'artist': 'Bob Dylan and The Band',
  'length (min)': 77,
  'year': 1975}]